In [19]:
import sys
sys.path.append('/home/maximilian.hofmann/ff_utils/src')

In [20]:
from importlib import reload
from itertools import repeat
from date_utils import partition_date_range
import pandas as pd

In [21]:
import functions as fu
reload(fu)

<module 'functions' from '/home/maximilian.hofmann/tools/pricing-tool-v2/data/functions.py'>

In [22]:
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [23]:
with open('/home/maximilian.hofmann/tools/brand_science_pricing/data/queries/sql_otter_data.sql', 'r') as f:
    sql_template = f.read()

In [24]:
project_name = 'css-operations'
stag_dataset_name = 'brand_science_stag'
stag_table_name = 'pricing_otter_data_stag'
full_stag_table_name = '.'.join([project_name, stag_dataset_name, stag_table_name])

In [25]:
begin_date = '2022-11-01'
end_date = '2023-01-01'
days_per_batch = 10

In [26]:
date_list = partition_date_range(pd.to_datetime(begin_date), pd.to_datetime(end_date), days_per_batch)
date_list

[['2022-10-31', '2022-11-09'],
 ['2022-11-10', '2022-11-19'],
 ['2022-11-20', '2022-11-29'],
 ['2022-11-30', '2022-12-09'],
 ['2022-12-10', '2022-12-19'],
 ['2022-12-20', '2022-12-26']]

In [27]:
min(cpu_count(), len(date_list))

6

In [28]:
country_code = 'CO'
clustering_fields = ['country_code']

In [29]:
num_processes = min(cpu_count(), len(date_list))
num_processes

6

In [30]:
res = []
with Pool(num_processes) as p:
    r = p.starmap(fu.process_otter_data_date_range, zip(repeat(sql_template), date_list, repeat(country_code), repeat(full_stag_table_name), repeat(clustering_fields)))
    res.append(r)

Start data cleaning...
Start data cleaning...
Start data cleaning...
Data cleaning finished!
Start data cleaning...
Job 353e4bf1-9392-483b-9ae8-6657d5e77951 is currently in state RUNNING
Job e8b65793-9c4a-4167-bb4c-6a59a88a9a7e is currently in state RUNNING
Data cleaning finished!
Job d7028881-93b4-4fa5-ac77-4b26c600171d is currently in state RUNNING
Data cleaning finished!
Job a90c47b6-c6f1-4128-88e6-92998dc3d9e7 is currently in state RUNNING
Job a90c47b6-c6f1-4128-88e6-92998dc3d9e7 is currently in state RUNNING
Data cleaning finished!
Data cleaning finished!
Job 8fbd4be4-0c04-47d9-8874-62c1a4e62b3f is currently in state RUNNING
Job d7b0425f-9423-441a-b624-a4d5dfb2b7e2 is currently in state RUNNING


In [31]:
prod_dataset_name = 'brand_science'
prod_table_name = 'pricing_otter_data'
full_prod_table_name = '.'.join([project_name, prod_dataset_name, prod_table_name])

In [32]:
create_tbl = fu.create_tbl(full_stag_table_name, full_prod_table_name)

In [33]:
upsert_tbl = fu.upsert_tbl_otter_data(full_stag_table_name, full_prod_table_name)